In [0]:
!pip install tensorflow-gpu
!pip install -q keras

     |████████████████████████████████| 516.2MB 31kB/s 


In [0]:
import os
from google.colab import drive

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from datetime import datetime
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D
from keras.layers import Layer, Dense, Dropout, Activation, Flatten, Reshape, Permute
from keras.layers import ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
import sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#from preprocessing.visualize_prepro import shiftedColorMap
import itertools
import tensorflow as tf

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
CODE_PATH_DIR = "drive/My Drive/Disertatie/"
GROUNDTRUTH_PATH = os.path.join(CODE_PATH_DIR, "Ground_truth")
TRAIN_IMAGES_PATH = os.path.join(CODE_PATH_DIR, "Train_Images")
TEST_IMAGES_PATH = os.path.join(CODE_PATH_DIR, "Test_Images")
OUTPUT_IMAGES_PATH = os.path.join(CODE_PATH_DIR, "Output_Images")

In [0]:
img_rows, img_cols = 600, 400
# output image dimensions
label_rows, label_cols = 600, 400

In [0]:
with tf.device('/device:GPU:0'):
    # we create two instances with the same arguments
    img_data_gen_args = dict(
        # featurewise_center=True,
        # featurewise_std_normalization=True,
        rescale=1. / 255,
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        fill_mode="constant",
        cval=0
    )
    label_data_gen_args = dict(
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        fill_mode="constant",
        cval=1
    )
    image_datagen = ImageDataGenerator(**img_data_gen_args)
    mask_datagen = ImageDataGenerator(**label_data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    # image_datagen.fit(images, augment=True, seed=seed)
    # mask_datagen.fit(masks, augment=True, seed=seed)

    image_generator = image_datagen.flow_from_directory(
        TRAIN_IMAGES_PATH,
        target_size=(img_rows, img_cols),
        class_mode=None,
        classes=None,
        batch_size=2,
        shuffle=False,
        seed=seed)

    mask_generator = mask_datagen.flow_from_directory(
        OUTPUT_IMAGES_PATH,
        target_size=(label_rows, label_cols),
        class_mode=None,
        classes=None,
        batch_size=2,
        shuffle=False,
        seed=seed)

  # combine generators into one which yields image and masks
    train_generator = zip(image_generator, mask_generator)

    checkpointer = ModelCheckpoint(filepath= CODE_PATH_DIR + "weights_v2.hdf5", verbose=1, save_best_only=False)
    model = load_model(CODE_PATH_DIR + 'spacenetmodel2.h5')
    model.compile(loss="binary_crossentropy", optimizer='adam',
                  metrics=['binary_accuracy'])
    model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=10,
        callbacks=[checkpointer])
    model.save(CODE_PATH_DIR + 'spacenetmodel2_v1.h5')

Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
Epoch 1/20
1000/1000 [==============================] - 1328s 1s/step - loss: -771.9414 - binary_accuracy: 1.3024e-05

Epoch 00001: saving model to drive/My Drive/Disertatie/weights_v2.hdf5
Epoch 2/20
1000/1000 [==============================] - 1299s 1s/step - loss: -772.7906 - binary_accuracy: 2.0910e-06

Epoch 00002: saving model to drive/My Drive/Disertatie/weights_v2.hdf5
Epoch 3/20
1000/1000 [==============================] - 1302s 1s/step - loss: -769.6711 - binary_accuracy: 7.1278e-06

Epoch 00003: saving model to drive/My Drive/Disertatie/weights_v2.hdf5
Epoch 4/20
1000/1000 [==============================] - 1300s 1s/step - loss: -771.3770 - binary_accuracy: 1.9667e-05

Epoch 00004: saving model to drive/My Drive/Disertatie/weights_v2.hdf5
Epoch 5/20
1000/1000 [==============================] - 1300s 1s/step - loss: -769.6201 - binary_accuracy: 1.0147e-05

Epoch 00005: saving model to drive/My Dri